# Kronicle — Quickstart / Integration Test

This notebook demonstrates how to use the `kronicle-sdk` to:

1. Create or upsert a channel via **KronicleWriter**
2. Insert rows of timeseries data
3. Read the channel metadata with **KronicleReader**
4. Read the rows back (as dict or Pandas DataFrame)
5. Perform simple verification assertions

Every created channel automatically receives a tag:

```
test: <now_utc>
```

This helps isolate test runs and later delete all test-tagged channels using KronicleSetup.

## Imports and Setup

Adjust the URL if your Kronicletorage API runs elsewhere.

In [ ]:
from models.iso_datetime import now_local
from utils.str_utils import tiny_id, uuid4_str

BASE_URL = "http://localhost:8000"  # Adjust if needed. The Kronicle server should of course be running.

## Instantiate a Writer connector

In [ ]:
from connectors.kronicle_writer import KronicleWriter

writer = KronicleWriter(BASE_URL)


## Scan the Kronicle server with the read abilities of the KronicleWriter


In [ ]:
main_channel_id, max_row_nb = writer.get_channel_with_max_rows()
print("Higher count of rows is",max_row_nb,"for channel", main_channel_id)
main_channel = writer.get_channel(main_channel_id)
print(writer.get_channel(main_channel_id))


## Review the data for this channel


In [ ]:
## Data as rows
writer.get_rows_for_channel(main_channel_id)



In [ ]:
## Data as columns
writer.get_cols_for_channel(main_channel_id)



## Prepare a unique test channel

We generate a fresh `sensor_id` for each run.

A unique **test tag** is added to the channel automatically:

In [ ]:
sensor_id = uuid4_str()
sensor_name = f"demo_channel_{tiny_id()}"
now_tag = now_local()

payload = {
    "sensor_id": sensor_id,
    "sensor_name": sensor_name,
    "sensor_schema": {"time": "datetime", "temperature": "float"},
    "metadata": {"unit": "°C"},
    "tags": {"test": now_tag},
    "rows": [
        {"time": "2025-01-01T00:00:00Z", "temperature": 12.3},
        {"time": "2025-01-01T00:01:00Z", "temperature": 12.8},
    ],
}

payload

## Create/Upsert the Channel + Insert Rows

In [ ]:
print('payload:', payload)
result = writer.insert_rows_and_upsert_channel(payload)
result

## Check the stored Metadata

In [ ]:
channel = writer.get_channel(sensor_id)
channel

## Read Rows (DataFrame)

In [ ]:
df = writer.get_rows_for_channel(sensor_id, return_type="df")
df

## Basic Assertions

These confirm both the SDK and the backend behaved as expected.

In [ ]:
assert len(df) == 2, "Should have exactly 2 rows"
assert abs(df["temperature"].iloc[0] - 12.3) < 1e-9
assert abs(df["temperature"].iloc[1] - 12.8) < 1e-9

print("✔ Basic read/write integration test succeeded.")

In [ ]:
from typing import Optional

from pydantic import BaseModel

from kronicle.models.iso_datetime import IsoDateTime, now
from kronicle.models.kronicable_sample import KronicableSample
from kronicle.models.kronicable_type import KronicableTypeChecker


# ------------------------------------------------------------
# Nested BaseModel example
# ------------------------------------------------------------
class MetaData(BaseModel):
    unit: str
    description: Optional[str] = None  # Optional field inside nested BaseModel

# ------------------------------------------------------------
# Main KronicableSample with optional primitives, nested BaseModels, list and dict
# ------------------------------------------------------------
class SensorSample(KronicableSample):
    timestamp: IsoDateTime
    temperature: Optional[float] = None            # Optional primitive
    meta: Optional[MetaData] = None                # Optional nested BaseModel
    tags: Optional[list[str]] = None               # Optional list of primitives
    extra: Optional[dict[str, MetaData]] = None    # Optional dict of BaseModels
    test_field: float | None = None
    test_meta: MetaData | None = None

# ------------------------------------------------------------
# Create sample instance with partial data
# ------------------------------------------------------------
sample = SensorSample(
    timestamp=now(),
    temperature=23.5,
    meta=MetaData(unit="°C"),                     # nested optional BaseModel
    tags=["room1", "test"],                       # optional list of primitives
    extra={"sensor1": MetaData(unit="°C", description="backup")}  # optional dict of BaseModels
)

# ------------------------------------------------------------
# Verify that KronicableTypeChecker correctly identifies optional fields
# ------------------------------------------------------------
for name, field in sample.model_fields.items():
    kt = KronicableTypeChecker(field.annotation)
    print(f"Field '{name}': valid={kt.is_valid()}, optional={kt.is_optional()}")

# ------------------------------------------------------------
# Convert the sample to a dictionary suitable for KroniclePayload
# Nested BaseModels, lists, and dicts should be serialized to JSON strings
# ------------------------------------------------------------
row_dict = sample.to_row()
print("\nSerialized row dictionary:")
print(row_dict)

# ------------------------------------------------------------
# Inspect the sensor schema generated by get_sensor_schema()
# Optional fields should appear as optional[...] in the schema
# ------------------------------------------------------------
schema = sample.get_sensor_schema()
print("\nGenerated sensor schema:")
print(schema)

## Proceed to Cleanup (via KronicleSetup)

Not implemented yet in this notebook — once `KronicleSetup` is ready, you will be able to automatically delete all channels containing the `test:` tag.

Example snippet to add later:
```python
# setup = KronicleSetup(BASE_URL)
# setup.delete_channels_by_tag("test", now_tag)
```